# 1. API Connection and Market Discovery Workflow

This notebook provides a comprehensive, step-by-step guide to connecting to the Capital.com API and discovering markets hierarchically. The goal is to retrieve all the detailed data for a specific instrument by the end of the process.

**Workflow Steps:**
1. Set up the environment and import necessary modules.
2. Initialize the API client with credentials.
3. Verify the successful connection by fetching session details.
4. Map the top-level market categories.
5. Drill down into a selected top-level category to view its sub-groups.
6. List the instruments within a sub-group.
7. Fetch all detailed data for a single instrument.
8. Close the connection.

### Step 1: Environment Setup and Imports

First, we import the necessary libraries and set up the Python environment to access our project's source code (the `src` directory) from the notebook. We also configure `loguru` and `pandas` for better readability.

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.capitalcom_bot.api_client import CapitalComAPIClient  # noqa: E402
from src.capitalcom_bot.config import load_api_credentials  # noqa: E402

logger.remove()
logger.add(sys.stderr, level="INFO")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

### Step 2: Initialize the API Client

We load the credentials from the `.env` file and create an instance of the `CapitalComAPIClient`. During initialization, the client automatically establishes an API session.

In [ ]:
api_creds = load_api_credentials()

client = CapitalComAPIClient(
    identifier=api_creds.identifier,
    password=api_creds.password,
    api_key=api_creds.api_key,
    demo_mode=True,
)

### Step 3: Verify the Connection

We fetch the details of the current session. If this cell runs without errors and returns data, it means the login was successful.

In [ ]:
try:
    session_details = client.get_session_details()
    print("Successfully fetched session details:")
    print(session_details.model_dump_json(indent=2))
except Exception as e:
    logger.error(f"Error fetching session details: {e}")

### Step 4: Map Top-Level Market Categories

We query all available top-level market categories (e.g., Indices, Shares) and load the result into an easy-to-use pandas DataFrame. Then, we programmatically find the ID for the 'indices' category.

In [ ]:
try:
    market_categories_response = client.get_market_categories()

    if market_categories_response.nodes:
        nodes_dict = [node.model_dump() for node in market_categories_response.nodes]
        df_categories = pd.DataFrame(nodes_dict)

        print("Top-level market categories in a DataFrame:")
        display(df_categories)
    else:
        logger.warning("No market categories were received.")

except Exception as e:
    logger.error(f"Error processing market categories: {e}", exc_info=True)

In [ ]:
if "df_categories" in locals():
    search_term = "indices"
    indices_category = df_categories[
        df_categories["name"].str.contains(search_term, case=False)
    ]

    if not indices_category.empty:
        display(indices_category)

        node_id_for_indices = indices_category["id"].iloc[0]
        print(f"\nThe ID for the 'indices' category is: '{node_id_for_indices}'")
    else:
        logger.warning(f"Category '{search_term}' not found.")

### Step 5: Drill Down into a Top-Level Category

Using the ID found previously, we query the content of the 'indices' category to see what sub-groups it contains.

In [ ]:
if "node_id_for_indices" in locals():
    try:
        logger.info(f"Querying content for category ID '{node_id_for_indices}'...")
        group_details = client.get_markets_by_category(node_id_for_indices)

        if group_details.get("nodes"):
            print("\nFound sub-groups:")
            df_subgroups = pd.DataFrame(group_details["nodes"])
            display(df_subgroups)
        else:
            logger.info("This category does not contain further sub-groups.")

        if group_details.get("markets"):
            print("\nIn the main category, there are also instruments listed directly:")
            display(pd.DataFrame(group_details["markets"]).head())

    except Exception as e:
        logger.error(f"Error querying category content: {e}", exc_info=True)
else:
    logger.warning(
        "The 'node_id_for_indices' variable does not exist. Please run the previous cell!"
    )

### Step 6: List Instruments in a Sub-Group

We select one of the sub-groups from the list above and list all the instruments it contains in a pandas DataFrame.

In [ ]:
if "df_subgroups" in locals() and not df_subgroups.empty:
    try:
        subgroup_to_query = df_subgroups.iloc[0]
        subgroup_id = subgroup_to_query["id"]
        subgroup_name = subgroup_to_query["name"]

        logger.info(
            f"Querying instruments for sub-group '{subgroup_name}' ({subgroup_id})..."
        )
        subgroup_content = client.get_markets_by_category(subgroup_id)

        if subgroup_content.get("markets"):
            df_instruments = pd.DataFrame(subgroup_content["markets"])
            print(f"\nInstruments in the '{subgroup_name}' sub-group:")
            display_columns = [
                "epic",
                "instrumentName",
                "bid",
                "offer",
                "marketStatus",
                "streamingPricesAvailable",
            ]
            existing_columns = [
                col for col in display_columns if col in df_instruments.columns
            ]
            display(df_instruments[existing_columns])
        else:
            logger.warning(f"The '{subgroup_name}' sub-group contains no instruments.")

    except Exception as e:
        logger.error(f"Error querying instruments in sub-group: {e}", exc_info=True)
else:
    logger.warning(
        "The 'df_subgroups' DataFrame does not exist or is empty. Please run the previous cell!"
    )

### Step 7: Fetch Detailed Data for a Single Instrument

Finally, we fetch all the detailed data for a single, specific instrument (e.g., US500) using its `epic` identifier and display it in an easy-to-read format.

In [ ]:
epic_to_query = "US500"

try:
    logger.info(f"Fetching detailed data for: '{epic_to_query}'...")
    full_details = client.get_full_market_details(epic=epic_to_query)
    details_dict = full_details.model_dump()
    df_details_flat = pd.json_normalize(details_dict)
    series_details = df_details_flat.T
    series_details.columns = ["Value"]

    print(f"\nDetailed data for the '{epic_to_query}' instrument:")
    display(series_details)

except Exception as e:
    logger.error(
        f"Error fetching details for epic '{epic_to_query}': {e}", exc_info=True
    )

### Step 8: Close the Session

At the end of the workflow, it's important to close the API session using the `close_session()` method. This performs a secure logout.

In [ ]:
client.close_session()